**Подключение библиотек и чтение данныx**

- *Токенизатор взят из библиотеки ***yargy*** *
- *Стоп-слова на русском взяты из ***nltk*** *
- *Регулярные выражения для очистки текста от знаков препинания и прочего мусора - библиотека ***re*** *
- *Для работы в датафреймом - ***pandas*** *
- *Метод преобразования слов в вектор, алгоритм LDA, KMeans и алгоритм уменьшения размерности PCA - ***scikit-learn*** *

In [925]:
from yargy.tokenizer import MorphTokenizer
from nltk.corpus import stopwords
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

df = pd.read_json('data/cintra_phoenix_oils_hr_mgck_feather.json').set_index('id')

**Определение токенизатора и множества стоп-слов на русском**

In [926]:
tokenizer = MorphTokenizer()
ru_stopwords = set(stopwords.words("russian"))

**В исходных текстах много различных знаков препринания, эмодзи и прочего "мусора". Проведем очистку данных - оставляем только слова и/или цифры, а также переводим слова в нижний регистр**

In [927]:
df = df['quote'].apply(lambda x: re.findall(r'\b\w+\b', str(x).lower())).to_frame()
df = df['quote'].apply(lambda x: ' '.join(x)).to_frame()

**Формирование токенов**

In [928]:
list_tok_2 = [[] for _ in range(len(df.quote.values))]
for i in range(len(df.quote.values)):
    list_tok = []
    tok = tokenizer(df.quote[i])
    for t in tok:
        if t.normalized not in ru_stopwords:
            list_tok.append(t.normalized)
    list_tok_2[i] = ' '.join(list_tok)
df['quote'] = list_tok_2

**Векторизация текста. Используется TfidfVectorizer**

In [929]:
vectorizer = TfidfVectorizer(max_df=0.6, min_df=5)
X = vectorizer.fit_transform([text for text in df.quote.values]).toarray()

**Уменьшение размерности**

In [930]:
pca = PCA(n_components=50)
X = pca.fit_transform(X)

**Выделение кластеров. Количество кластеров - это гиперпараметр, в решении формируется пять кластеров**

In [931]:
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

list_cluster = []
for cluster_idx in range(num_clusters):
    cluster_documents = [df.quote.values[i] for i, label in enumerate(kmeans.labels_) if label == cluster_idx]
    list_cluster.append(cluster_documents)

**Использование алгоритма LDA для анализа полученных кластеров. Устанавливается количество кластеров равных единице, так как наш текст уже разбит на кластеры и нам нужно уже к текстам в кластере найти те слова, которые полно описывают рассматриваемый кластер**

In [932]:
num_topics = 1
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)

for j in range(len(list_cluster)):
    vectorizer = TfidfVectorizer(max_df=0.07, min_df=1)
    X = vectorizer.fit_transform([text for text in list_cluster[j]]).toarray()
    lda.fit(X)
    for topic_idx, topic_words in enumerate(lda.components_):
        top_words_idx = topic_words.argsort()[-10:][::-1]
        top_words = [vectorizer.get_feature_names_out()[i] for i in top_words_idx]
        print(f"Тема для кластера № {j + 1}: {', '.join(top_words)}")

Тема для кластера № 1: форма, делать, колдун, стать, квартал, качество, слышать, свой, огонь, давать
Тема для кластера № 2: цк, огненный, стать, день, каждый, феникс, зачаровать, приходиться, зарплата, делать
Тема для кластера № 3: колонка, бандит, далёкий, равно, помочь, случиться, замечательный, пара, супер, сотрудник
Тема для кластера № 4: чан, будуть, расчитывать, чен, арений, ууйти, предложить, зпл, зна, телепортировать
Тема для кластера № 5: друг, ребёнок, информация, отдельный, прийти, нужный, добираться, кофе, зрыть, вериться


**В результате формируются определенное количество тем и список слов, которые описывают эти темы**

Настраиваемые параметры 
 - *max_df* - параметр векторизатора, процент самых частоповторяющихся слов, которые удаляются
 - *min_df* - параметр векторизатора, порог удаления самых непопулярных слов
 - *num_clusters* - параметр кластеризатора KMeans, количество кластеров

max_df в последнем vectorizer такой низкий, так как в датасете много слов, которые не несут важной информации. 

В итоге получилось пять кластеров
 1. Что-то про качество, форму, а также про временные промежутки (год, квартал). Похоже форму приходится менять каждый год или квартал, так как у нее плохое качество 
 2. Фигурируют слова "условие", "дракон", "поход". Так как в названиях фентезийных компаний часто содержится слово "Дракон", видимо текста описывают условия работы в компаниях. 
 3. Фигурируют слова "бандит", "помочь", "случиться", "сотрудник". Похоже бандиты представляют серьезную опасность для сотрудников
 4. Фигурируют слова "зарплата", "дело", "ночь", "час". Похоже речь идет про ночные смены, которые оплачиваются каждый час
 5. Много раз фигурировали слова "ребенок" и "друг". Также встречались слова "добираться", "прийти", "нужный", "информация". Возможно речь про нужную информацию, которую нужно срочно передать кому-то хорошо знакомому. 